 # Práctica  5.
 ----
  
  Universidad : UTEC \\
  Curso       : Inteligencia Artificial \\
  Profesor    : Cristian López Del Alamo \\
  Tema        : Árboles de Decisión \\
  

 ----

 Nombres y  Apellidos de Integrantes: (No olvide poner el % de participacion)
 - Integrante 1: Adrian Auqui (100 %)
 - Integrante 2: Miguel Yurivilca (100 %)
 - Integrante 3: Moises Chambilla (100%)
 - Integrante 4: Javier Chavarria (100 %)

*Una vez concluya la práctica debe subir el link de su colab a este  [Drive]
(https://docs.google.com/spreadsheets/d/1XCxGVmf8g29C7RZSPOqxvZHjjRZg45LHWR8mkZOfS1o/edit?usp=drivesdk)*


Para este ejercicio, usted deberá contruir su propio árbol de desición.
- Tenga en cuenta que se trata de un algoritmo recursivo.
- El caso base se da cuanto todos los elementos de un nodo tiene las mismas etiquetas, es decir, es un nodo terminal. Luego, el label de ese nodo toma el valor de la etiqueta común.
- En el caso que no sea un nodo terminal, el algoritmo debe buscar uno de los
feactures por el cual dividirse y para esto use Ganacia de Información (Entropía o Gini).
- Divida el dataset usando el feacture que genere una mayor ganancia de información en el padre o un menor GINI y llame recursivamente a la función create_DT.

Usted usará la base de datos iris, con 4 características y 3 clases.
Tome aleatoriamente 80% de los datos para crear el árbol y el resto para
probar el accuracy de la predicción.
Finalmente, muestra mediante una matriz de confusión el **accuracy** de su modelo.

Trabaje en equipo:

[Link de apoyo 1](https://towardsdatascience.com/the-simple-math-behind-3-decision-tree-splitting-criterions-85d4de2a75fe)

[Link de apoyo 2](https://www.quantstart.com/articles/Beginners-Guide-to-Decision-Trees-for-Supervised-Machine-Learning/)



In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

# Asume que 'Iris.csv' está en la raíz de tu Google Drive
path = "./iris_dataset.csv"
iris = pd.read_csv(path)
print(iris.head())

class Nodo:

    def __init__(self, X, Y, index):
        self.m_X = X
        self.m_Y = Y
        self.m_Index = index
        self.m_HijoIzquierdo = None
        self.m_HijoDerecho = None
        self.m_Feature = None
        self.m_Valor = None

    def IsTerminal(self, Y):

        return len(np.unique(Y)) == 1

    def BestSplit(self, X, Y):

        H = self.Entropy(Y)
        for i in range(X.shape[1]):
            feature_values = np.unique(X[:, i])
            for value in feature_values:
                X_izquierdo = X[X[:, i] <= value]
                Y_izquierdo = Y[X[:, i] <= value]
                H_izquierdo = self.Entropy(Y_izquierdo)

                X_derecho = X[X[:, i] > value]
                Y_derecho = Y[X[:, i] > value]
                H_derecho = self.Entropy(Y_derecho)
                H_condicional = (len(Y_izquierdo) / len(Y)) * H_izquierdo + (len(Y_derecho) / len(Y)) * H_derecho
                if H - H_condicional > self.m_Hoja:
                    self.m_Hoja = H - H_condicional
                    self.m_Feature = i
                    self.m_Valor = value

        return self.m_Feature, self.m_Valor

    def Entropy(self, Y):
        p = np.unique(Y, return_counts=True)[1] / len(Y)
        H = 0
        for pi in p:
            if pi != 0:
                H -= pi * np.log2(pi)
        return H

class DT:

    def __init__(self, X, Y, index):
        self.m_X = X
        self.m_Y = Y
        self.m_Index = index
        self.m_Root = None

    def create_DT(self):

        self.m_Root = Nodo(self.m_X, self.m_Y, 0)

        if self.m_Root.IsTerminal(self.m_Y):
            return self.m_Root
        feature, valor = self.m_Root.BestSplit(self.m_X, self.m_Y)
        self.m_Root.m_HijoIzquierdo = Nodo(
            self.m_X[self.m_X[:, feature] <= valor],
            self.m_Y[self.m_X[:, feature] <= valor],
            self.m_Index + 1,
        )
        self.m_Root.m_HijoDerecho
        return self.m_Root
    
    def predict(self, X):
        return np.array([self.predict_sample(x) for x in X])
    
    def predict_sample(self, x):
        current_node = self.m_Root
        while current_node.m_Feature is not None:
            if x[current_node.m_Feature] <= current_node.m_Valor:
                current_node = current_node.m_HijoIzquierdo
            else:
                current_node = current_node.m_HijoDerecho
        return np.unique(current_node.m_Y)[0]
    

X = iris.drop(columns=['species']).values
Y = iris['species'].values
dt = DT(X, Y, 0)
dt.create_DT()
print(dt.predict(X))

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

        target  
0  Iris-setosa  
1  Iris-setosa  
2  Iris-setosa  
3  Iris-setosa  
4  Iris-setosa  


KeyError: "['species'] not found in axis"